In [ ]:
import os
import requests
import numpy as np
import rasterio
from rasterio.transform import from_bounds
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from itertools import combinations

# ------------------------------------------------------------
# Folders
data_in_geoserver_folder = "../geoserver_data/data"
os.makedirs(data_in_geoserver_folder, exist_ok=True)

# ------------------------------------------------------------
# WMS layers
layers = {
    "pm10": "https://wms.zh.ch/AwelLHPM10JahreZHWMS",
    "pm25": "https://wms.zh.ch/AwelLHPM25JahreZHWMS",
    "no2": "https://wms.zh.ch/AwelLHNO2JahreZHWMS",
    "ozon": "https://wms.zh.ch/AwelLHMP98JahreZHWMS",
    "russ": "https://wms.zh.ch/ImmissionenZHWMS",
    "laerm": "https://wms.geo.admin.ch/"
}

year = 2023
russ_year = 2020
layer_names = {
    "pm10": f"pm10-jahre-{year}",
    "pm25": f"pm25-jahre-{year}",
    "no2": f"no2-jahre-{year}",
    "ozon": f"mp98-jahre-{year}",
    "russ": f"bc-{russ_year}",
    "laerm": "ch.bafu.laerm-strassenlaerm_tag"
}

# Range thresholds for normalization
range_per_layer = {
    "pm10": [0,26],
    "pm25": [0,14],
    "no2": [0,60],
    "ozon": [0,170],
    "russ": [0,2],
    "laerm": [0,70]
}

bbox = (2654500, 1222400, 2720000, 1300000)
width, height = 1600, 1200

# ------------------------------------------------------------
# Fetch and normalize all layers
layer_arrays = {}
for key, wms_url in layers.items():
    params = {
        "SERVICE": "WMS",
        "VERSION": "1.3.0",
        "REQUEST": "GetMap",
        "LAYERS": layer_names[key],
        "CRS": "EPSG:2056",
        "BBOX": f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}",
        "WIDTH": width,
        "HEIGHT": height,
        "FORMAT": "image/tiff"
    }
    r = requests.get(wms_url, params=params)
    r.raise_for_status()
    raw_tif_path = os.path.join(data_in_geoserver_folder, f"{key}_raw.tif")
    with open(raw_tif_path, "wb") as f:
        f.write(r.content)

    # Read TIFF
    with rasterio.open(raw_tif_path) as src:
        arr = src.read(1)
        crs = src.crs

    # Mask valid data
    mask = arr != 255
    arr_norm = np.zeros_like(arr, dtype=float)
    low, high = range_per_layer[key]
    arr_norm[mask] = np.clip(arr[mask], low, high)  # clip
    arr_norm[mask] = (arr_norm[mask] - low) / (high - low) * 40  # normalize to fuzzy_max=40
    arr_norm[~mask] = np.nan
    layer_arrays[key] = arr_norm
    print(f"✅ {key} layer downloaded and normalized.")

# ------------------------------------------------------------
# Create fuzzy cards per layer
fuzzy_layers = {}
fuzzy_max = 40
for key in layer_arrays:
    arr_norm = layer_arrays[key]
    low_threshold, high_threshold = 15, 25  # can also adjust per layer

    pm = ctrl.Antecedent(np.linspace(0, fuzzy_max, 100), key)
    pm['good'] = fuzz.trimf(pm.universe, [0, 0, low_threshold])
    pm['moderate'] = fuzz.trimf(pm.universe, [low_threshold, (low_threshold+high_threshold)/2, high_threshold])
    pm['bad'] = fuzz.trimf(pm.universe, [high_threshold, fuzzy_max, fuzzy_max])

    quality = ctrl.Consequent(np.linspace(0,100,100), f'{key}_quality')
    quality['very_good'] = fuzz.trimf(quality.universe, [0,0,25])
    quality['good'] = fuzz.trimf(quality.universe, [20,40,60])
    quality['liveable'] = fuzz.trimf(quality.universe, [50,60,70])
    quality['bad'] = fuzz.trimf(quality.universe, [65,80,90])
    quality['very_bad'] = fuzz.trimf(quality.universe, [85,100,100])

    # Rules per layer
    rules = [
        ctrl.Rule(pm['bad'], quality['very_bad']),
        ctrl.Rule(pm['moderate'], quality['liveable']),
        ctrl.Rule(pm['good'], quality['very_good']),
    ]
    fuzzy_ctrl = ctrl.ControlSystem(rules)

    h, w = arr_norm.shape
    fuzzy_output = np.full((h, w), np.nan, dtype=float)
    for i in range(h):
        for j in range(w):
            if not np.isnan(arr_norm[i,j]):
                sim = ctrl.ControlSystemSimulation(fuzzy_ctrl)
                sim.input[key] = arr_norm[i,j]
                try:
                    sim.compute()
                    fuzzy_output[i,j] = sim.output[f'{key}_quality']
                except:
                    fuzzy_output[i,j] = np.nan
    fuzzy_layers[key] = fuzzy_output
    print(f"✅ Fuzzy card created for {key}")

# ------------------------------------------------------------
# Multi-input fuzzy quality map
# Antecedents: one per pollutant
antecedents = {}
for key in fuzzy_layers:
    antecedents[key] = ctrl.Antecedent(np.linspace(0,100,100), key)
    antecedents[key]['very_bad'] = fuzz.trimf(antecedents[key].universe,[85,100,100])
    antecedents[key]['bad'] = fuzz.trimf(antecedents[key].universe,[65,80,90])
    antecedents[key]['liveable'] = fuzz.trimf(antecedents[key].universe,[50,60,70])
    antecedents[key]['good'] = fuzz.trimf(antecedents[key].universe,[20,40,60])
    antecedents[key]['very_good'] = fuzz.trimf(antecedents[key].universe,[0,0,25])

# Consequent
quality = ctrl.Consequent(np.linspace(0,100,100),'quality')
quality['very_good'] = fuzz.trimf(quality.universe, [0,0,25])
quality['good'] = fuzz.trimf(quality.universe, [20,40,60])
quality['liveable'] = fuzz.trimf(quality.universe, [50,60,70])
quality['bad'] = fuzz.trimf(quality.universe, [65,80,90])
quality['very_bad'] = fuzz.trimf(quality.universe, [85,100,100])

# ------------------------------------------------------------
# Multi-layer rules
rules = []

vars_list = list(fuzzy_layers.keys())

# 1️⃣ VERY BAD: at least 2 inputs are 'very_bad'
for combo in combinations(vars_list, 2):
    condition = antecedents[combo[0]]['very_bad'] & antecedents[combo[1]]['very_bad']
    rules.append(ctrl.Rule(condition, quality['very_bad']))

# 2️⃣ BAD: exactly one is 'bad'
for var in vars_list:
    rules.append(ctrl.Rule(antecedents[var]['bad'], quality['bad']))

# 3️⃣ VERY GOOD: all six are 'good'
condition = antecedents[vars_list[0]]['good']
for var in vars_list[1:]:
    condition = condition & antecedents[var]['good']
rules.append(ctrl.Rule(condition, quality['very_good']))

# 4️⃣ GOOD: at least 3 are 'good'
for combo in combinations(vars_list,3):
    condition = antecedents[combo[0]]['good'] & antecedents[combo[1]]['good'] & antecedents[combo[2]]['good']
    rules.append(ctrl.Rule(condition, quality['good']))

# 5️⃣ LIVEABLE: default fallback
all_good = antecedents[vars_list[0]]['good']
for var in vars_list[1:]:
    all_good = all_good | antecedents[var]['good']
rules.append(ctrl.Rule(~all_good, quality['liveable']))

quality_ctrl = ctrl.ControlSystem(rules)

# ------------------------------------------------------------
# Compute final quality map
h, w = next(iter(fuzzy_layers.values())).shape
final_quality = np.full((h, w), np.nan, dtype=float)

for i in range(h):
    for j in range(w):
        sim = ctrl.ControlSystemSimulation(quality_ctrl)
        for key in vars_list:
            sim.input[key] = fuzzy_layers[key][i,j] if not np.isnan(fuzzy_layers[key][i,j]) else 50
        try:
            sim.compute()
            final_quality[i,j] = sim.output['quality']
        except:
            final_quality[i,j] = np.nan

# ------------------------------------------------------------
# Save final TIFF
output_path = os.path.join(data_in_geoserver_folder,"zurich_fuzzy_quality_final.tif")
transform = from_bounds(bbox[0], bbox[1], bbox[2], bbox[3], w, h)

with rasterio.open(
    output_path,
    "w",
    driver="GTiff",
    height=h,
    width=w,
    count=1,
    dtype=final_quality.dtype,
    crs=crs,
    transform=transform
) as dst:
    dst.write(final_quality, 1)

print(f"✅ Final fuzzy environmental quality map saved: {output_path}")

# ------------------------------------------------------------
# Plot with correct colors
norm_output = (final_quality - np.nanmin(final_quality)) / (np.nanmax(final_quality) - np.nanmin(final_quality))
classes = np.zeros_like(norm_output)
classes[(norm_output < 0.2)] = 1
classes[(norm_output >= 0.2) & (norm_output < 0.4)] = 2
classes[(norm_output >= 0.4) & (norm_output < 0.6)] = 3
classes[(norm_output >= 0.6) & (norm_output < 0.8)] = 4
classes[(norm_output >= 0.8)] = 5

cmap = ListedColormap(["darkgreen", "lightgreen", "yellow", "orange", "red"])
plt.figure(figsize=(10,10))
plt.imshow(classes, cmap=cmap, extent=[bbox[0], bbox[2], bbox[1], bbox[3]])
plt.title("Fuzzy Environmental Quality Map of Zürich")
plt.xlabel("Easting (EPSG:2056)")
plt.ylabel("Northing (EPSG:2056)")
plt.colorbar(ticks=[1,2,3,4,5], label="1=Very Good → 5=Very Bad")
plt.show()


✅ pm10 layer downloaded and normalized.
✅ pm25 layer downloaded and normalized.
✅ no2 layer downloaded and normalized.
✅ ozon layer downloaded and normalized.
✅ russ layer downloaded and normalized.
✅ laerm layer downloaded and normalized.
✅ Fuzzy card created for pm10
✅ Fuzzy card created for pm25
✅ Fuzzy card created for no2
✅ Fuzzy card created for ozon
✅ Fuzzy card created for russ
✅ Fuzzy card created for laerm
